<a target="_blank" href="https://colab.research.google.com/github/Moikii/SHEL">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!git clone https://github.com/Moikii/SHEL.git

fatal: destination path 'SHEL' already exists and is not an empty directory.


In [20]:
%%capture
!pip install ultralytics

import sys
import os
from IPython.utils import io
from ultralytics import YOLO
py_file_location = "/content/SHEL/src"
sys.path.append(os.path.abspath(py_file_location))
import dataset_generation_functions as dgf
import photo_preparation_functions as ppf

In [38]:
# input variables

create_new_dataset = True
small_dataset_bool = True

# only when creating a new dataset
number_of_images = 1000
max_number_of_cards_per_image = 4
min_size = 0.2
max_size = 0.7
max_rotation = 360
overlapping = True
seed = 42

# model training parameters
epochs = 2

In [39]:
# do not change!
backgrounds_id = '1KHAYHo9voKRUWtff5ES07hBRUmT7vSg5'
photos_ids = ['1o0f3q5zJzfRXs9vSMb1uR9KcUfWtFItZ', '1Ot7kAToB1YTSraS3hgbw9zBHDmHtZhpj']
dataset_ids = ['11cMzPUd_ETyOzzb0iJMtONWP10Nu7Zwc', '1o6z8eRo0pKg9IXQR80-E8p7XvwQMXLwX']

In [40]:
def download_data_from_drive(ID, folder_name):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$ID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$ID -O data.zip && rm -rf /tmp/cookies.txt
    with io.capture_output() as captured:
      !unzip data.zip -d './'$folder_name
      !rm data.zip


def create_new_dataset(small_dataset_bool, number_of_images, max_number_of_cards_per_image,
                       min_size, max_size, rotation, overlapping, seed):
  download_data_from_drive(photos_ids[small_dataset_bool], 'photos')
  download_data_from_drive(backgrounds_id, 'backgrounds')

  PLAYING_CARDS_DIR = ppf.process_photos('/content/photos')
  DATASET_DIR = dgf.generate_dataset('/content/backgrounds', PLAYING_CARDS_DIR, '/content/dataset',
                                     number_of_images, max_number_of_cards_per_image, min_size,
                                     max_size, max_rotation, overlapping, seed)
  return DATASET_DIR


def load_dataset(small_dataset_bool):
  download_data_from_drive(dataset_ids[small_dataset_bool], 'dataset')
  DATASET_DIR = '/content/dataset'
  return DATASET_DIR


def train_model(create_new_dataset, small_dataset_bool, epochs, number_of_images,
                max_number_of_cards_per_image, min_size, max_size, rotation, overlapping, seed):
  if create_new_dataset:
    DATASET_DIR = create_new_dataset(small_dataset_bool, number_of_images, max_number_of_cards_per_image,
                                     min_size, max_size, rotation, overlapping, seed)
  else:
    DATASET_DIR = load_dataset(small_dataset_bool)

  model = YOLO('yolov8n.pt')
  results = model.train(data = DATASET_DIR + '/data.yaml', epochs = epochs)


In [44]:
!rm -rf runs
!rm -rf dataset
!rm -rf photos
!rm -rf photos_processed
!rm -rf sample_data
!rm -rf backgrounds